In [35]:
import pandas as pd

In [36]:
data=pd.read_csv("D:/ML/House_Price_Prediction/notebooks/data/Housing.csv")

In [37]:
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [38]:
data.columns

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [39]:
X=data.drop(labels=['price'],axis=1)

In [40]:
y=data['price']

In [33]:
cat_column=data.columns[data.dtypes=='O']

In [41]:
X=data.drop(labels=['price'],axis=True)

In [42]:
y=data['price']

In [43]:
num_column=X.columns[X.dtypes!='O']

In [44]:
cat_column

Index(['mainroad', 'guestroom', 'basement', 'hotwaterheating',
       'airconditioning', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [45]:
num_column

Index(['area', 'bedrooms', 'bathrooms', 'stories', 'parking'], dtype='object')

In [46]:
mainroad=['no','yes']
guestroom=['no','yes']
basement=['no','yes']
hotwaterheating=['no','yes']
airconditioning=['no','yes']
prefarea=['no','yes']
furnishingstatus=['unfurnished','semi-furnished','furnished']

In [47]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [48]:
num_pipeline=Pipeline(
    
    steps=[
        ('imputer',SimpleImputer()),
        ('scaling',StandardScaler())
    ]
)

In [49]:
cat_pipeline=Pipeline(
    
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('encoding',OrdinalEncoder(categories=[mainroad,basement,guestroom,airconditioning,
                                               hotwaterheating,prefarea,furnishingstatus]))
    ]
)

In [50]:
preprocessor=ColumnTransformer(
[
    ('num',num_pipeline,num_column),
    ('cat',cat_pipeline,cat_column)
]

)

In [170]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.25,random_state=21)

In [171]:
preprocessor.fit_transform(X_train)

array([[-1.04606989, -1.33526978, -0.54779844, ...,  0.        ,
         0.        ,  0.        ],
       [-0.48637237,  0.03354949, -0.54779844, ...,  1.        ,
         0.        ,  1.        ],
       [ 0.35907789, -1.33526978, -0.54779844, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.24335983, -1.33526978, -0.54779844, ...,  0.        ,
         0.        ,  1.        ],
       [-0.38954707,  0.03354949,  1.41274333, ...,  0.        ,
         0.        ,  0.        ],
       [-1.00356122,  0.03354949, -0.54779844, ...,  1.        ,
         0.        ,  1.        ]])

In [172]:
preprocessor.transform(X_test)

array([[ 0.17723523,  0.03354949,  1.41274333, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.44173364, -1.33526978, -0.54779844, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.8857131 ,  0.03354949,  1.41274333, ...,  1.        ,
         0.        ,  2.        ],
       ...,
       [ 0.66844656,  1.40236876,  1.41274333, ...,  1.        ,
         0.        ,  1.        ],
       [-0.91854388, -1.33526978,  1.41274333, ...,  0.        ,
         0.        ,  1.        ],
       [ 2.23654425,  0.03354949, -0.54779844, ...,  0.        ,
         0.        ,  1.        ]])

In [173]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [174]:
X_train.head()

,num__area,num__bedrooms,num__bathrooms,num__stories,num__parking,cat__mainroad,cat__guestroom,cat__basement,cat__hotwaterheating,cat__airconditioning,cat__prefarea,cat__furnishingstatus
0,-1.046070,-1.335270,-0.547798,-0.962697,-0.768230,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.486372,0.033549,-0.547798,0.185784,-0.768230,0.0,1.0,1.0,0.0,1.0,0.0,1.0
2,0.359078,-1.335270,-0.547798,-0.962697,0.388367,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.701277,1.402369,-0.547798,0.185784,-0.768230,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-1.071575,0.033549,-0.547798,1.334264,-0.768230,1.0,0.0,0.0,0.0,0.0,1.0,2.0


In [175]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import numpy as np

In [176]:
def model_evaluate(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rsme=np.sqrt(mean_squared_error(true,predicted))
    r_square=r2_score(true,predicted)

    return mae,mse,rsme,r_square    

In [177]:
models={
    'LinearRegression': LinearRegression(),
    'ridge':Ridge(),
    'Lasso':Lasso(),
    'ElasticNet':ElasticNet(),
}

In [178]:
model_list=[]
r2_list=[]

In [179]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    
    #model prediction
    y_pred=model.predict(X_test)
    
    mae,mse,rsme,r_square=model_evaluate(y_test,y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('model training performance')
    print("mean_absolute_error : ",mae)
    print('mean_squared_error : ',mse)
    print("rmse : ",rsme)
    print('r_square : ',r_square*100)
    
    r2_list.append(r_square)

    print('-'*30)
    print('\n')

LinearRegression
model training performance
mean_absolute_error :  831425.7485335297
mean_squared_error :  1304416046096.425
rmse :  1142110.347600627
r_square :  63.163878448915234
------------------------------


ridge
model training performance
mean_absolute_error :  831192.7092143768
mean_squared_error :  1306680751848.042
rmse :  1143101.3742656608
r_square :  63.09992417865487
------------------------------


Lasso
model training performance
mean_absolute_error :  831425.6178371554
mean_squared_error :  1304418316353.4575
rmse :  1142111.3414870976
r_square :  63.1638143378793
------------------------------


ElasticNet
model training performance
mean_absolute_error :  855894.130849307
mean_squared_error :  1496271972319.1343
rmse :  1223221.963635028
r_square :  57.745953516310365
------------------------------




In [5]:
from src.HousePricePrediction.logger import logging

In [6]:
logging.info("hey")